Assumptions, the environment for running this notebook was arrived at by building a dedicated docker file.

https://cloud.docker.com/repository/registry-1.docker.io/russelljarvis/nuo

You can run use dockerhub to get the appropriate file, and launch this notebook using Kitematic.

# Import libraries
To keep the standard running version of minimal and memory efficient, not all available packages are loaded by default. In the cell below I import a mixture common python modules, and custom developed modules associated with NeuronUnit (NU) development

In [1]:

import numpy as np
import os
import pickle
import pandas as pd
from neuronunit.tests.fi import RheobaseTestP
from neuronunit.optimization.model_parameters import reduced_dict, reduced_cells  
from neuronunit.optimization import optimization_management as om
from sciunit import scores# score_type 

from neuronunit.optimization.data_transport_container import DataTC
from neuronunit.tests.fi import RheobaseTestP# as discovery
from neuronunit.optimization.optimization_management import dtc_to_rheo, format_test, nunit_evaluation
import quantities as pq
from neuronunit.models.reduced import ReducedModel
from neuronunit.optimization.model_parameters import model_params, path_params
LEMS_MODEL_PATH = path_params['model_path']
list_to_frame = []
from neuronunit.tests.fi import RheobaseTestP

# The Izhiketich model is instanced using some well researched parameter sets.


First lets get the points in parameter space, that Izhikich himself has published about. These points are used by other scientists and the [Open Source Brain project](http://opensourcebrain.org/projects/izhikevichmodel?explorer=https%3A%2F%2Fraw.githubusercontent.com%2FOpenSourceBrain%2FIzhikevichModel%2Fsample%2FosbSessions%2Fsample%2FSample_Session.json)  to establish between model reproducibility. The itial motivating factor for choosing these points as constellations, of all possible parameter space subsets, is that these points where initially tuned and used as best guesses for matching real observed experimental recordings.

In [2]:
explore_param = {k:(np.min(v),np.max(v)) for k,v in reduced_dict.items()}

## Get the experimental Data pertaining to four different classes or neurons, that can constrain models.
Next we get some electro physiology data for four different classes of cells that are very common targets of scientific neuronal modelling. We are interested in finding out what are the most minimal, and detail reduced, low complexity model equations, that are able to satisfy 

Below are some of the data set ID's I used to query neuroelectro.
To save time for the reader, I prepared some data earlier to save time, and saved the data as a pickle, pythons preferred serialisation format.

The interested reader can find some methods for getting cell specific ephys data from neuroelectro in a code file (neuronunit/optimization/get_neab.py) 


In [3]:
purkinje ={"id": 18, "name": "Cerebellum Purkinje cell", "neuron_db_id": 271, "nlex_id": "sao471801888"}
fi_basket = {"id": 65, "name": "Dentate gyrus basket cell", "neuron_db_id": None, "nlex_id": "nlx_cell_100201"}
pvis_cortex = {"id": 111, "name": "Neocortex pyramidal cell layer 5-6", "neuron_db_id": 265, "nlex_id": "nifext_50"}
#This olfactory mitral cell does not have datum about rheobase, current injection values.
olf_mitral = {"id": 129, "name": "Olfactory bulb (main) mitral cell", "neuron_db_id": 267, "nlex_id": "nlx_anat_100201"}
ca1_pyr = {"id": 85, "name": "Hippocampus CA1 pyramidal cell", "neuron_db_id": 258, "nlex_id": "sao830368389"}
pipe = [ fi_basket, ca1_pyr, purkinje,  pvis_cortex]

In [4]:
electro_tests = []
obs_frame = {}
test_frame = {}

from neuronunit.optimization import get_neab
try: 

    electro_path = str(os.getcwd())+'all_tests.p'

    assert os.path.isfile(electro_path) == True
    with open(electro_path,'rb') as f:
        (obs_frame,test_frame) = pickle.load(f)

except:
    for p in pipe:
        p_tests, p_observations = get_neab.get_neuron_criteria(p)
        obs_frame[p["name"]] = p_observations#, p_tests))
        test_frame[p["name"]] = p_tests#, p_tests))
    electro_path = str(os.getcwd())+'all_tests.p'
    with open(electro_path,'wb') as f:
        pickle.dump((obs_frame,test_frame),f)

# Cast the tabulatable data to pandas data frame
There are many among us who prefer potentially tabulatable data to be encoded in pandas data frame.

In [5]:
for k,v in test_frame.items():
    if "olf_mit" not in k:
        obs = obs_frame[k]
        v[0] = RheobaseTestP(obs['Rheobase'])
df = pd.DataFrame.from_dict(obs_frame)
print(test_frame.keys())

dict_keys(['Dentate gyrus basket cell', 'Hippocampus CA1 pyramidal cell', 'Neocortex pyramidal cell layer 5-6', 'Cerebellum Purkinje cell'])


In the data frame below, you can see many different cell types

In [6]:
df


,Cerebellum Purkinje cell,Dentate gyrus basket cell,Hippocampus CA1 pyramidal cell,Neocortex pyramidal cell layer 5-6
Cell Capacitance,"{'mean': 620.2725 pF, 'n': 4, 'std': 261.30303...",NaN,"{'mean': 89.7960714285714 pF, 'n': 14, 'std': ...","{'mean': 150.584166666667 pF, 'n': 12, 'std': ..."
Input Resistance,"{'mean': 142.057692307692 Mohm, 'n': 13, 'std'...","{'mean': 222.475 Mohm, 'n': 4, 'std': 168.2911...","{'mean': 107.080327644332 Mohm, 'n': 113, 'std...","{'mean': 120.672073643411 Mohm, 'n': 86, 'std'..."
Membrane Time Constant,NaN,"{'mean': 13.25 ms, 'n': 2, 'std': 3.25 ms}","{'mean': 24.5021946169772 ms, 'n': 46, 'std': ...","{'mean': 15.7342424242424 ms, 'n': 33, 'std': ..."
Resting membrane potential,"{'mean': -61.5916666666667 mV, 'n': 6, 'std': ...","{'mean': -62.3 mV, 'n': 3, 'std': 4.0472212689...","{'mean': -65.2261863636364 mV, 'n': 110, 'std'...","{'mean': -68.2481434599156 mV, 'n': 79, 'std':..."
Rheobase,"{'mean': 680.794444444444 pA, 'n': 3, 'std': 4...","{'mean': 106.7 pA, 'n': 1, 'std': 0.0 pA}","{'mean': 189.24 pA, 'n': 17, 'std': 287.163664...","{'mean': 213.849583333333 pA, 'n': 32, 'std': ..."
Spike Amplitude,"{'mean': 71.2308333333333 mV, 'n': 6, 'std': 9...","{'mean': 78.6 mV, 'n': 2, 'std': 5.7 mV}","{'mean': 86.364525297619 mV, 'n': 64, 'std': 1...","{'mean': 80.4351020408164 mV, 'n': 49, 'std': ..."
Spike Half-Width,"{'mean': 0.41412962962963 ms, 'n': 9, 'std': 0...","{'mean': 0.9975 ms, 'n': 4, 'std': 0.442175021...","{'mean': 1.31895278450363 ms, 'n': 59, 'std': ...","{'mean': 1.20769387755102 ms, 'n': 49, 'std': ..."
Spike Threshold,"{'mean': -46.8947619047619 mV, 'n': 7, 'std': ...","{'mean': -38.4 mV, 'n': 2, 'std': 3.4 mV}","{'mean': -47.5985714285714 mV, 'n': 70, 'std':...","{'mean': -42.7357232704403 mV, 'n': 53, 'std':..."


In [7]:
df['Hippocampus CA1 pyramidal cell']

# enable R style caching.
#!pip install git+https://github.com/rossant/ipycache
#%load_ext ipycache

Cell Capacitance              {'mean': 89.7960714285714 pF, 'n': 14, 'std': ...
Input Resistance              {'mean': 107.080327644332 Mohm, 'n': 113, 'std...
Membrane Time Constant        {'mean': 24.5021946169772 ms, 'n': 46, 'std': ...
Resting membrane potential    {'mean': -65.2261863636364 mV, 'n': 110, 'std'...
Rheobase                      {'mean': 189.24 pA, 'n': 17, 'std': 287.163664...
Spike Amplitude               {'mean': 86.364525297619 mV, 'n': 64, 'std': 1...
Spike Half-Width              {'mean': 1.31895278450363 ms, 'n': 59, 'std': ...
Spike Threshold               {'mean': -47.5985714285714 mV, 'n': 70, 'std':...
Name: Hippocampus CA1 pyramidal cell, dtype: object

# Tweak Izhikitich equations
with educated guesses based on information that is already encoded in the predefined experimental observations.

In otherwords use information that is readily amenable into hardcoding into equations 

Select out the 'Neocortex pyramidal cell layer 5-6' below, as a target for optimization

In [8]:
free_params = ['a','b','k','c','C','d','vPeak','vr']
hc_ = reduced_cells['RS']
hc_['vr'] = -65.2261863636364
hc_['vPeak'] = hc_['vr'] + 86.364525297619
explore_param['C'] = (hc_['C']-20,hc_['C']+20)
explore_param['vr'] = (hc_['vr']-5,hc_['vr']+5)
use_test = test_frame["Neocortex pyramidal cell layer 5-6"]

#for t in use_test[::-1]:
#    t.score_type = scores.RatioScore
test_opt = {}

with open('data_dump.p','wb') as f:
    pickle.dump(test_opt,f)

In [9]:

use_test[0].observation
print(use_test[0].name)

rtp = RheobaseTestP(use_test[0].observation)
use_test[0] = rtp
print(use_test[0].observation)

RheobaseTestP
{'mean': array(213.849583333333) * pA, 'n': 32, 'std': array(170.452454715608) * pA}


The code in the next two cells below is used to create a sparse grid sampling over the parameter space, using the published and well corrobarated parameter points, from Izhikitch publications, and the Open Source brain

In [10]:
reduced_cells.keys()
test_frame.keys()


dict_keys(['Dentate gyrus basket cell', 'Hippocampus CA1 pyramidal cell', 'Neocortex pyramidal cell layer 5-6', 'Cerebellum Purkinje cell'])

In [12]:
df = pd.DataFrame(index=list(test_frame.keys()),columns=list(reduced_cells.keys()))

try:
    assert 1==2
    with open('Izh_seeds.p','rb') as f:
        seeds = pickle.load(f)
    #assert seeds is not None

except:
    # print('exceptional circumstances pickle file does not exist, rebuilding sparse grid for Izhikich')
    # Below we perform a sparse grid sampling over the parameter space, using the published and well corrobarated parameter points, from Izhikitch publications, and the Open Source brain, shows that without optimization, using off the shelf parameter sets to fit real-life biological cell data, does not work so well.

    for k,v in reduced_cells.items():
        temp = {}
        temp[str(v)] = {}
        dtc = DataTC()
        dtc.tests = use_test
        dtc.attrs = v
        dtc.backend = 'RAW'
        dtc.cell_name = 'vanilla'
        for key, use_test in test_frame.items():
            dtc.tests = use_test
            dtc = dtc_to_rheo(dtc)
            dtc = format_test(dtc)
            if dtc.rheobase is not None:
                if dtc.rheobase!=-1.0:
                    dtc = nunit_evaluation(dtc)
            df[k][str(key)] = dtc.get_ss()

    best_params = {}
    for index, row in df.iterrows():
        best_params[index] = row == row.min()
        best_params[index] = best_params[index].to_dict()


    seeds = {}
    for k,v in best_params.items():
        for nested_key,nested_val in v.items():
            if True == nested_val:
                seed = reduced_cells[nested_key]
                seeds[k] = seed
    with open('Izh_seeds.p','wb') as f:
        pickle.dump(seeds,f)




exceptional circumstances pickle file does not exist, rebuilding sparse grid for Izhikich


/opt/conda/lib/python3.5/site-packages/sciunit/scores/complete.py:68: RuntimeWarning: divide by zero encountered in true_divide
  value = (p_value - o_mean)/o_std
/home/jovyan/neuronunit/neuronunit/tests/passive.py:85: RuntimeWarning: overflow encountered in exp
  vm_fit[offset:,0] = a * np.exp(-t[offset:]/b) + c
/home/jovyan/neuronunit/neuronunit/tests/passive.py:85: RuntimeWarning: overflow encountered in multiply
  vm_fit[offset:,0] = a * np.exp(-t[offset:]/b) + c


As I said above: 'A sparse grid sampling over the parameter space, using the published and well corrobarated parameter points, from Izhikitch publications, and the Open Source brain', The output from the grid shows that without optimization, using off the shelf parameter sets to fit real-life biological cell data, does not work so well.

In [13]:
df

,RS,IB,LTS,TC,TC_burst
Dentate gyrus basket cell,5.33575,6.44702,6.03769,6.27535,6.38901
Hippocampus CA1 pyramidal cell,4.2652,5.84565,5.70527,5.74633,6.06554
Neocortex pyramidal cell layer 5-6,4.66967,5.83884,6.21074,6.36435,6.5461
Cerebellum Purkinje cell,6.04342,5.95173,5.47237,5.69825,5.53656


As can be seen none of these well researched parameter sets correspond to good fits, with experimental data about waveform shape. These bad fit's warrant the use of an optimizer, which can efficiently, sparsely, and intelligently explore the parameter space for better model fits. None of the models score higher than:

```4.2652```, and the best is ```5.47237``` in the specific case of the cerebellum purkinje cell. 
and None of the first guess parameter set were very good for the cerebellum Purkinje cell. It should be easy to find a model parameterization that scores better than ```5.47```. A first guess. 

The best NeuronUnit score values found via this sparse grid sampling, are used to seed the first generation of the genetic algorithm, as one of the great motivators of GA optimization, is not to waste any computation time.


We now optimize using it to show what can be achieved with the GA.

In [22]:


try:
    with open('multi_objective_izhi.p','rb') as f:
        test_opt = pickle.load(f)
except:
    MU = 6
    NGEN = 150


    seed = seeds[key]
    print(seed)
    ga_out, _ = om.run_ga(explore_param,NGEN,use_test,free_params=free_params, NSGA = True, MU = MU,seed_pop = seed, model_type = str('RAW'))

    test_opt =  {str('multi_objective_izhi')+str('_ga_out'):ga_out}
    with open('multi_objective_izhi.p','wb') as f:
        pickle.dump(test_opt,f)




In [37]:
for k,v in test_opt.items():
    best_model = v['pf'][0]
summed = best_model.dtc.get_ss()
print('its {2} that {0} is significantly less than {1}'.format(summed,5.47,summed<5.47))
print(key)

its True that 3.1302660755008325 is significantly less than 5.47
Cerebellum Purkinje cell


In [42]:
key = str('Neocortex pyramidal cell layer 5-6')


try:
    with open('multi_objective_izhi'+str(key)+'.p','rb') as f:
        test_opt = pickle.load(f)
except:
    MU = 6
    NGEN = 150
    seed = seeds[key]
    use_test = test_frame[key]
    print(seed)
    ga_out, _ = om.run_ga(explore_param,NGEN,use_test,free_params=free_params, NSGA = True, MU = MU,seed_pop = seed, model_type = str('RAW'))

    test_opt =  {str('multi_objective_izhi')+str(key):ga_out}
    with open('multi_objective_izhi'+str(key)+'.p','wb') as f:
        pickle.dump(test_opt,f)



{'vt': -40, 'c': -50, 'vr': -65.2261863636364, 'vPeak': 21.138338933982595, 'C': 100, 'd': 100, 'k': 0.7, 'a': 0.03, 'b': -2}


/home/jovyan/neuronunit/neuronunit/tests/passive.py:85: RuntimeWarning: overflow encountered in exp
  vm_fit[offset:,0] = a * np.exp(-t[offset:]/b) + c
/home/jovyan/neuronunit/neuronunit/tests/passive.py:85: RuntimeWarning: overflow encountered in multiply
  vm_fit[offset:,0] = a * np.exp(-t[offset:]/b) + c
INFO:__main__:gen	nevals	avg    	std     	min    	max    
1  	7     	5.22203	0.821745	3.72429	6.03463
2  	5     	5.21212	0.735291	3.97883	6.03463


true minimum 3.72428861926


INFO:__main__:3  	4     	5.13677	0.626946	4.25394	5.65035


true minimum 3.72428861926


...

true minimum 2.52307142918


INFO:__main__:149	6     	3.87063	0.657622 	2.80581	4.47962


true minimum 2.52307142918


INFO:__main__:150	6     	3.95304	0.490059 	3.23952	4.40974


true minimum 2.52307142918


The optimizer needs to find a parameter set much better than 4.66 in order to be worthwhile, for the case of the Neo Cortical Pyramidal neuron. In only 10 generations the optimizer finds a parameter set 3.24. In 23 generations it finds a model with score 2.99.

In [77]:
for k,v in test_opt.items():
    #list_best = []
    best_model = v['pf'][0]
    second_best_model = v['pf'][1]
    third_best_model = v['pf'][2]
    list_best = [best_model,second_best_model,third_best_model]
summed = best_model.dtc.get_ss()
print('its {2} that {0} is significantly less than {1}'.format(summed,5.47,summed<5.47))
print(key)

its True that 2.523071429176899 is significantly less than 5.47
Neocortex pyramidal cell layer 5-6


In [44]:
best_model.dtc.attrs

{'C': 80.32327976522879,
 'a': 0.017035973642268745,
 'b': -1.995763503718458,
 'c': -56.110284353607987,
 'd': 39.870524431534278,
 'k': 0.52366419898375982,
 'vPeak': 3.4684930072840432,
 'vr': -67.290036991963049}

In [45]:
best_model.dtc.scores


{'CapacitanceTest': 0.14163892773420128,
 'InjectedCurrentAPAmplitudeTest': 0.9999999997243523,
 'InjectedCurrentAPThresholdTest': 0.3846682140841715,
 'InjectedCurrentAPWidthTest': 0.10023627982283978,
 'InputResistanceTest': 0.08782389566447724,
 'RestingPotentialTest': 0.02152619567192071,
 'RheobaseTestP': 0.7321282418074668,
 'TimeConstantTest': 0.05504967466746935}

In [50]:
best_model.dtc.score

{'CapacitanceTest': {'value': 0.8583610722657987},
 'InjectedCurrentAPAmplitudeTest': {'value': 2.7564772686616834e-10},
 'InjectedCurrentAPThresholdTest': {'value': 0.6153317859158285},
 'InjectedCurrentAPWidthTest': {'value': 0.8997637201771602},
 'InputResistanceTest': {'value': 0.9121761043355228},
 'RestingPotentialTest': {'value': 0.9784738043280793},
 'RheobaseTestP': {'value': 0.26787175819253317},
 'TimeConstantTest': {'value': 0.9449503253325306}}

In [49]:
for row in test_frame['Neocortex pyramidal cell layer 5-6']:
    print(row.observation)


{'mean': array(213.849583333333) * pA, 'n': 32, 'std': array(170.452454715608) * pA}
{'mean': array(120.672073643411) * Mohm, 'n': 86, 'std': array(77.6331608333564) * Mohm}
{'mean': array(15.7342424242424) * ms, 'n': 33, 'std': array(7.31162636832495) * ms}
{'mean': array(150.584166666667) * pF, 'n': 12, 'std': array(139.683884626343) * pF}
{'mean': array(-68.2481434599156) * mV, 'n': 79, 'std': array(6.53234788156637) * mV}
{'mean': array(1.20769387755102) * ms, 'n': 49, 'std': array(0.534345918375033) * ms}
{'mean': array(80.4351020408164) * mV, 'n': 49, 'std': array(12.7488030357545) * mV}
{'mean': array(-42.7357232704403) * mV, 'n': 53, 'std': array(8.04073233409085) * mV}


In [51]:
key = str('Neocortex pyramidal cell layer 5-6')



'''
dtc = dtc_to_rheo(dtc)
dtc = format_test(dtc)
if dtc.rheobase is not None:
    if dtc.rheobase!=-1.0:
        dtc = nunit_evaluation(dtc)
'''        
    


In [76]:
#df

In [83]:
from neuronunit.optimization.optimization_management import bridge_judge
dfs = []
for model in list_best:
    df = pd.DataFrame(index=list(dtc.tests),columns=['observation','prediction','disagreement'])#,columns=list(reduced_cells.keys()))

    dtc = DataTC()
    dtc.tests = use_test
    dtc.attrs = model.dtc.attrs
    dtc.backend = 'RAW'
    dtc.cell_name = 'vanilla'
    #for key, use_test in test_frame.items():
    dtc.tests = test_frame[key]
    dtc = dtc_to_rheo(dtc)
    dtc = format_test(dtc)
    for k,t in enumerate(dtc.tests):
        t.params = dtc.vtest[k]
        score,pred = bridge_judge((t,dtc))
        #print(score,pred)       
        df.iloc[k]['observation'] = t.observation['mean'] 
        try:
            agreement = np.abs(t.observation['mean'] - pred['value'])
            df.iloc[k]['prediction'] = pred['value']
            df.iloc[k]['disagreement'] = agreement

        except:
            agreement = np.abs(t.observation['mean'] - pred['mean'])
            df.iloc[k]['prediction'] = pred['mean']
            df.iloc[k]['disagreement'] = agreement
    dfs.append(df)
result = pd.concat(dfs)
result         

,observation,prediction,disagreement
RheobaseTestP,213.849583333333 pA,24.9921875 pA,188.857395833333 pA
InputResistanceTest,120.672073643411 Mohm,129234556.37052076 kg*m**2/(s**3*A**2),8.562482727109753 Mohm
TimeConstantTest,15.7342424242424 ms,0.016239105584981928 s,0.5048631607395286 ms
CapacitanceTest,150.584166666667 pF,1.2565606321596948e-10 s**4*A**2/(kg*m**2),24.92810345069752 pF
RestingPotentialTest,-68.2481434599156 mV,-0.06842440161605753 V,0.17625815614194096 mV
InjectedCurrentAPWidthTest,1.20769387755102 ms,0.001275 s,0.06730612244898015 ms
InjectedCurrentAPAmplitudeTest,80.4351020408164 mV,-3.394359574052097e-05 V,80.46904563655693 mV
InjectedCurrentAPThresholdTest,-42.7357232704403 mV,-0.03869543745051891 V,4.040285819921387 mV
RheobaseTestP,213.849583333333 pA,25.154296875 pA,188.695286458333 pA
InputResistanceTest,120.672073643411 Mohm,127521728.72962098 kg*m**2/(s**3*A**2),6.849655086209978 Mohm


In [ ]:
df
from neuronunit.models.reduced import ReducedModel
from neuronunit.optimization.data_transport_container import DataTC
from neuronunit.optimization.model_parameters import model_params, path_params, transcribe_units
from neuronunit.optimization.optimization_management import mint_generic_model
LEMS_MODEL_PATH = path_params['model_path']


model = ReducedModel(LEMS_MODEL_PATH,name = str('vanilla'),backend = ('RAW'))
dtc = DataTC()
dtc.attrs = model.attrs